In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl


In [2]:
class ray:
    def __init__(self, x, y, dx, dy, xLim, yLim):
        '''(x,y) posiiton
           (dx, dy) velocity'''
        self.xLim = xLim
        self.yLim = yLim
        self.x = x
        self.y = y

        self.sourceX = x - dx
        self.sourceY = y - dy

        self.distanceFromSource = 0

        mag = (dx**2 + dy**2)**(1/2)
        self.dx = dx/mag
        self.dy = dy/mag

        #self.point, = ax.plot(x,y, 'go', markersize = .5)

    def move(self):
        scale = 0.3
        self.x = (self.x + self.dx*scale)
        self.y = (self.y + self.dy*scale)
        self.distanceFromSource = ((self.x - self.sourceX)**2+(self.y - self.sourceY)**2)**(1/2)
        self.color= mpl.colormaps["cool"](self.distanceFromSource/5)

        remove = False
        if abs(self.x) > self.xLim:
            #self.point.remove()
            remove = True
        elif abs(self.y) > self.yLim:
            #self.point.remove()
            remove = True
        return remove


In [3]:
class source:

    def __init__(self, x, y, ax, xLim, yLim):
        self.radius = .1
        self.x = x
        self.y = y
        self.ax = ax
        self.xLim = xLim
        self.yLim = yLim
    
    def plotSource(self):
        '''draws the circle'''
        #offset to rotate circle so no lines parallel to x or y axis
        offset = np.pi/16

        #start and stop of theta parameter
        self.thetaStart = 0 + offset
        self.thetaEnd = np.pi * 2 + offset

        #number of nodes on source     ///change name numsamples to numnodes
        self.numSamples = 13

        self.nodePhotonStorageList = []
        for _ in range(self.numSamples):
            self.nodePhotonStorageList.append([])
        linSpace = np.linspace(self.thetaStart, self.thetaEnd, self.numSamples)
        xList = []
        yList = []
        self.raysList = []
        for theta in linSpace:
            xList.append(self.radius*np.cos(theta) + self.x)
            yList.append(self.radius*np.sin(theta) + self.y)
            raysPlot, = self.ax.plot(xList.copy(), yList.copy(), 'g', markersize = 0.4)
            self.raysList.append(raysPlot)
        
        #plot the nodes
        self.sourcePlot, = self.ax.plot(xList, yList, 'bo', markersize = 0.1)
        
    
    def moveSource(self, newX, newY):
        linSpace = np.linspace(self.thetaStart, self.thetaEnd, self.numSamples)
        xList = []
        yList = []
        self.x = newX
        self.y = newY
        for theta in linSpace:
            xList.append(self.radius*np.cos(theta) + self.x)
            yList.append(self.radius*np.sin(theta) + self.y)
        self.sourcePlot.set_xdata(xList)
        self.sourcePlot.set_ydata(yList)
        

    
    def emit(self):

        #copyRayList = self.nodePhotonStorageList.copy()
        c = 0
        for node in self.nodePhotonStorageList:
            for thisRay in node:
                if(thisRay.move()):
                    #copyRayList[c].remove(thisRay)
                    self.nodePhotonStorageList[c].remove(thisRay)
            c += 1
                    
        #self.nodePhotonStorageList = copyRayList

        linSpace = np.linspace(self.thetaStart, self.thetaEnd, self.numSamples)
        c = 0
        for node in linSpace:
            x = self.radius*np.cos(node) + self.x
            y = self.radius*np.sin(node) + self.y
            dx = self.radius*np.cos(node)
            dy = self.radius*np.sin(node)
            newRay = ray(x, y, dx, dy, self.xLim, self.yLim)
            self.nodePhotonStorageList[c].append(newRay)
            c+=1
        
        c = 0
        for node in self.nodePhotonStorageList:
            xList = []
            yList = []
            for photon in node:
                xList.append(photon.x)
                yList.append(photon.y)
            
            self.raysList[c].set_xdata(xList)
            self.raysList[c].set_ydata(yList)
            #self.ax.plot(xList,yList, 'ro')
            c += 1


            
        
            


In [4]:
import matplotlib.animation as animation
%matplotlib qt

fig, ax = plt.subplots()

sourceX = 1
sourceY = 0

xLim = 10

yLim = 10

sour = source(sourceX, sourceY, ax, xLim, yLim)



ax.set_xlim(-xLim, xLim)
ax.set_ylim(-yLim, yLim)

sour.plotSource()





t = 0
dt = .2
def update(frame):
    global sourceX, sourceY, t, dt

    sour.emit()
    sourceX = np.cos(t)
    sourceY = np.sin(t*.8)
    sour.moveSource(sourceX, sourceY)

    t+= dt



ani = animation.FuncAnimation(fig=fig, func=update)

plt.show()

C:\Users\jshen\AppData\Local\Temp\ipykernel_23160\3887940525.py:40: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig=fig, func=update)
